# Demo
The demo is a Python-based Jupyter notebook which contains:
- an example of how to add, retrieve, and delete a document using the Elasticsearch Python Client
- an example showing how a document persists even if the containers are thrown away
- a very basic intro to Kibana

## 1. Check the containers

Confirm that Elasticsearch and Kibana are available from the host machine:
- To see Elasticsearch and the existing indices, open a new tab and go to http://localhost:8002/_cat/indices?v.
    - Based on the index names, you can see that Kibana saves its data inside Elasticsearch even though it operates in its own container.
- To see Kibana, open a new tab and go to http://localhost:8003/.

## 2. Add, retrieve, and delete a document in Elasticsearch
- Connect to the Elasticsearch container by creating an authenticated [Python Elasticsearch client](https://elasticsearch-py.readthedocs.io/en/v7.11.0/)

In [1]:
from datetime import datetime
# the build process installed the `elasticsearch` package from `requirements.txt` when it created the Jupyter container
from elasticsearch import Elasticsearch

In [2]:
from dotenv import dotenv_values

In [3]:
# dictionary of variables from .env
config = dotenv_values("../config/.env") 

In [4]:
es = Elasticsearch(hosts=[{"host": "elasticsearch"}], http_auth=(config['ELASTIC_USERNAME'], config['ELASTIC_PASSWORD']))

- Add a document to the `demo` index and give it an `id` of 1. Elasticsearch will automatically create the `demo` index if it doesn't already exist.

In [5]:
# Example from https://elasticsearch-py.readthedocs.io/en/v7.11.0/index.html
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="demo", id=1, body=doc)
print(res['result'])

created


- Go to http://localhost:8002/_cat/indices?v. You should see a new index called `demo` with one document in it.
```
health status index                           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   demo                            Ilxkmf2yQCWLxIZVOIleFg   1   1          1            0      4.7kb          4.7kb
```

- Retrieve the document based on its `id`

In [6]:
res = es.get(index="demo", id=1)
print(res['_source'])

{'author': 'kimchy', 'text': 'Elasticsearch: cool. bonsai cool.', 'timestamp': '2021-10-20T03:54:56.937133'}


- Use a search query and match all documents in the `demo` index

In [7]:
res = es.search(index="demo", body={"query": {"match_all": {}}})
print("Got %d Hits" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 1 Hits
2021-10-20T03:54:56.937133 kimchy: Elasticsearch: cool. bonsai cool.


- Delete the document

In [8]:
es.delete(index="demo", id=1)

{'_index': 'demo',
 '_type': '_doc',
 '_id': '1',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

- There might be a time delay between when the document is deleted and when it is reflected in the search. Refresh the index to update it.

In [9]:
es.indices.refresh(index="demo")

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [10]:
# search again to confirm the document is gone
res = es.search(index="demo", body={"query": {"match_all": {}}})
print("Got %d Hits" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 0 Hits


- Delete the `demo` index

In [11]:
es.indices.delete(index='demo')

{'acknowledged': True}

## 3. Check that documents persist even after the containers are thrown away
- Add another document to Elasticsearch and check that it exists

In [12]:
doc = {
    'author': 'jovyan',
    'text': 'document which should persist',
    'timestamp': datetime.now(),
}
res = es.index(index="demo-persist", id=2, body=doc)
print(res['result'])

created


In [13]:
res = es.get(index="demo-persist", id=2)
print(res['_source'])

{'author': 'jovyan', 'text': 'document which should persist', 'timestamp': '2021-10-20T03:55:15.845087'}


- Run `docker-compose down` in your host machine's terminal
- Use `docker ps -a` to confirm that all containers from the multi-container environment have been thrown away
- Run `docker-compose up -d` to spin up the containers again
- Open this notebook:
    - skip all the cells above and run the cell below to check that the document you added still exists
    - If you see 'ConnectionRefusedError', try again in a few seconds (the Elasticsearch service is likely still booting up)

In [1]:
from dotenv import dotenv_values
config = dotenv_values("../config/.env") 
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=[{"host": "elasticsearch"}], http_auth=(config['ELASTIC_USERNAME'], config['ELASTIC_PASSWORD']))
res = es.get(index="demo-persist", id=2)
print(res['_source'])

{'author': 'jovyan', 'text': 'document which should persist', 'timestamp': '2021-10-20T03:55:15.845087'}


- delete the `demo-persist` index

In [2]:
es.indices.delete(index='demo-persist')

{'acknowledged': True}

## 4. Explore your data in Kibana
- When you go to http://localhost:8003/, you'll see the homepage for Kibana.
- Click `Kibana: Visualize and Analyze` and then `Add your data`.
- Since you don't have any data yet, let's adds some!

In [3]:
import time

In [4]:
def add_test_docs(number_of_docs):
    for n in range(number_of_docs):
        doc = {
        'author': 'jovyan',
        'text': f'this is the text for document {n}',
        'timestamp': datetime.now(),
        }
        es.index(index="demo-kibana", id=n, body=doc)
        print(f"added document {n}")
        time.sleep(1)

In [5]:
add_test_docs(10)

added document 0
added document 1
added document 2
added document 3
added document 4
added document 5
added document 6
added document 7
added document 8
added document 9


In [6]:
es.indices.refresh(index="demo-kibana")

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [7]:
res = es.search(index="demo-kibana", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 10 Hits:
2021-10-20T03:57:36.465497 jovyan: this is the text for document 0
2021-10-20T03:57:37.775005 jovyan: this is the text for document 1
2021-10-20T03:57:38.790308 jovyan: this is the text for document 2
2021-10-20T03:57:39.822712 jovyan: this is the text for document 3
2021-10-20T03:57:40.836326 jovyan: this is the text for document 4
2021-10-20T03:57:41.853791 jovyan: this is the text for document 5
2021-10-20T03:57:42.873192 jovyan: this is the text for document 6
2021-10-20T03:57:43.888203 jovyan: this is the text for document 7
2021-10-20T03:57:44.911346 jovyan: this is the text for document 8
2021-10-20T03:57:45.930949 jovyan: this is the text for document 9


- Click `Check for new data` and `Create index pattern`
- Set the index pattern name to `demo-kibana` and the Time field to `timestamp`
- Open up the side menu (three lines in the top left corner)
- Click the `Analytics: Discover` tab. You should be able to see the 10 documents you added sorted by `timestamp`.
- Feel free to add more test data to explore how the `Analytics: Visualize` tab can help you monitor your data collection. 
    - The [kibana-enhanced-table plugin](https://github.com/fbaligand/kibana-enhanced-table) has been added to the Kibana container so you can find "enhanced table" and "document table" under the aggregation-based visualization type.
- After you build 1+ visualizations, you can put your them together using `Analytics: Dashboard`.

When you're done,

1. delete the `demo-kibana` index in Elasticsearch

In [8]:
es.indices.delete(index='demo-kibana')

{'acknowledged': True}

2. and delete the index pattern in Kibana by clicking:
- the Elastic icon to return to the home page (top left corner)
- `Manage`
- `Kibana: Index Patterns` (bottom left)
- `demo-kibana` under the `Pattern` column
- and finally the trash can icon to remove it